In [ ]:
import nltk
from nltk.corpus import brown
from sklearn.svm import SVC
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Đảm bảo bạn đã tải về các gói dữ liệu của NLTK
nltk.download('brown')
nltk.download('universal_tagset')

# Tải dữ liệu từ Brown Corpus với nhãn từ loại theo bộ nhãn Universal Tagset
sentences = brown.tagged_sents(tagset='universal')[:1000]
#print(len(sentences))
#sentences = sentences[:1000]

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


In [ ]:
print(sentences[0])

[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')]


In [ ]:
# Định nghĩa hàm để trích xuất đặc trưng từ mỗi từ và ngữ cảnh của nó
def features(sentence, index):
    """ Trả về một dict mô tả các đặc trưng của từ và ngữ cảnh của nó """
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'is_all_caps': sentence[index].upper() == sentence[index],
        'is_all_lower': sentence[index].lower() == sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word_is_one_char': len(sentence[index - 1]) == 1 if index > 0 else False,
    }

In [ ]:
# Tạo dữ liệu đặc trưng và nhãn
X, y = [], []
for sent in sentences:
    untagged = [word for word, tag in sent]
    for i, (word, tag) in enumerate(sent):
        X.append(features(untagged, i))
        y.append(tag)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction import DictVectorizer

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tạo pipeline bao gồm vector hóa và mô hình Logistic Regression
#model = make_pipeline(DictVectorizer(sparse=False), LogisticRegression(max_iter=10))

# Bước 1: Vector hóa dữ liệu
vectorizer = DictVectorizer(sparse=False)
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)
print(X_train_transformed.shape)

# Bước 2: Huấn luyện mô hình Logistic Regression
model = LogisticRegression(max_iter=10)
model.fit(X_train_transformed, y_train)

# Huấn luyện mô hình
model.fit(X_train_transformed, y_train)

# Đánh giá mô hình
y_pred = model.predict(X_test_transformed)
print("Accuracy:", accuracy_score(y_test, y_pred))


(17663, 15730)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Accuracy: 0.8922101449275363


In [ ]:
print(X[0])

{'word': 'The', 'is_first': True, 'is_last': False, 'prev_word': '', 'next_word': 'Fulton', 'is_capitalized': True, 'is_all_caps': False, 'is_all_lower': False, 'prefix-1': 'T', 'prefix-2': 'Th', 'prefix-3': 'The', 'suffix-1': 'e', 'suffix-2': 'he', 'suffix-3': 'The', 'prev_word_is_one_char': False}
